# DNN Critic model

In [1]:
import pandas as pd 
import numpy as np  
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'
from collections import Counter
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from keras import optimizers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

# read data
mydt = pd.read_csv("../dataset/KDD-2018/trainr2l.csv")
print(mydt['Label'].value_counts())

# Create matrix of features and matrix of target variable 
dataset = mydt.values
X = dataset[:,0:41]
y = dataset[:,41]

# convert the scaled array to dataframe 
min_max_scaler = MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)

X_full = X_scale
y_full = y

X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.3, random_state=1)

print(Counter(y_train))
print(Counter(y_test))

# define the keras model
model = Sequential()
model.add(Dense(128, input_shape=(41,), activation='relu'))
model.add(Dense(41, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# compile the keras model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Building Critic WGAN model...")
# fit the keras model on the dataset
mod = model.fit(X_train, y_train, epochs=30, batch_size=100, verbose=1, validation_data=(X_test, y_test))

# make class predictions with the model
y_pred = (model.predict(X_test) > 0.5).astype(int)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: %.2f%%" % (accuracy * 100))

# Save model
# serialize model to JSON
model_json = model.to_json()
with open("DNN_critic_model_KDD_2.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("DNN_critic_model_KDD_2.h5")
print("Saved model!")

0    14019
1      506
Name: Label, dtype: int64
Counter({0.0: 9809, 1.0: 358})
Counter({0.0: 4210, 1.0: 148})
Building Critic WGAN model...
Epoch 1/30


2022-09-03 14:56:00.050497: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


102/102 [==============================] - 2s 6ms/step - loss: 0.1625 - accuracy: 0.9647 - val_loss: 0.0680 - val_accuracy: 0.9660
Epoch 2/30
102/102 [==============================] - 1s 6ms/step - loss: 0.0487 - accuracy: 0.9856 - val_loss: 0.0346 - val_accuracy: 0.9911
Epoch 3/30
102/102 [==============================] - 1s 7ms/step - loss: 0.0353 - accuracy: 0.9917 - val_loss: 0.0294 - val_accuracy: 0.9920
Epoch 4/30
102/102 [==============================] - 1s 5ms/step - loss: 0.0308 - accuracy: 0.9925 - val_loss: 0.0258 - val_accuracy: 0.9931
Epoch 5/30
102/102 [==============================] - 1s 7ms/step - loss: 0.0273 - accuracy: 0.9928 - val_loss: 0.0237 - val_accuracy: 0.9940
Epoch 6/30
102/102 [==============================] - 1s 5ms/step - loss: 0.0244 - accuracy: 0.9931 - val_loss: 0.0217 - val_accuracy: 0.9945
Epoch 7/30
102/102 [==============================] - 1s 7ms/step - loss: 0.0224 - accuracy: 0.9939 - val_loss: 0.0202 - val_accuracy: 0.9945
Epoch 8/30
102/10

In [2]:
# Testing 
cm = confusion_matrix(y_test, y_pred) 
tnr = recall_score(y_test, y_pred) 
fpr = 1 - tnr

print ("confusion_matrix for test data\n",cm)
print(classification_report(y_test,y_pred,labels=np.unique(y_pred)))
print('Accuracy: %.4f' % accuracy_score(y_test, y_pred))
print('Precision: %.4f' % precision_score(y_test, y_pred))
print('Recall: %.4f' % recall_score(y_test, y_pred))
print('F1 Score: %.4f' % f1_score(y_test, y_pred))
print('FPR Score: %.4f' % fpr)

confusion_matrix for test data
 [[4203    7]
 [   6  142]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4210
           1       0.95      0.96      0.96       148

    accuracy                           1.00      4358
   macro avg       0.98      0.98      0.98      4358
weighted avg       1.00      1.00      1.00      4358

Accuracy: 0.9970
Precision: 0.9530
Recall: 0.9595
F1 Score: 0.9562
FPR Score: 0.0405


# WGAN

In [ ]:
import torch 
import torch.nn
import torch.nn.functional as nn 
import torch.autograd as autograd 
import torch.optim as optim 
import torch.utils.data as Data
from torch.autograd import Variable
import numpy as np 
import tensorflow as tf 
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler 
from sklearn.metrics import confusion_matrix 
import keras
from keras.models import Sequential 
from keras.layers import Dense 
from keras import optimizers
from keras.models import model_from_json
import matplotlib 
matplotlib.use('agg') 
import matplotlib.pyplot as plt 
import matplotlib.gridspec as gridspec 

# hyperparameters 
mb_size = 64 
z_dim = 100 
X_dim = 41 
y_dim = 1 
h_dim = 128 #hidden 
lr = 0.00005 #learning_rate

# read data
df = pd.read_csv("../dataset/KDD-2018/trainr2l.csv")
print('original data label features number:')
print(df['Label'].value_counts())

# data preparation
criteria0 = df['Label'] == 1 #filter attack data 
data0 = df[criteria0]

float_array = data0.values.astype(float) 
scaler = MinMaxScaler()
scaled_array = scaler.fit_transform(float_array)
attack_data = pd.DataFrame(scaled_array)
attack_data.columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'Label']

print('----------------------------------------') 
print('attack value number:') 
print(attack_data['Label'].value_counts())

# transform data to tensor format
train_target = torch.tensor(attack_data['Label'].values.astype(np.float32)) 
train = torch.tensor(attack_data.drop(['Label'], axis = 1).values.astype(np.float32)) 
train_tensor = Data.TensorDataset(train, train_target) 

#creat DataLoader with batch and shuffle
train_loader = torch.utils.data.DataLoader(dataset = train_tensor, batch_size = mb_size, shuffle = True)

# define models
# Generator
G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim), 
    torch.nn.Sigmoid()
)

# Discriminator
D = torch.nn.Sequential( 
    torch.nn.Linear(X_dim, h_dim), 
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim), 
    torch.nn.ReLU(),
    # No sigmoid
    torch.nn.Linear(h_dim, 1),
)

# setup cpu using cpu (if use gpu -> set cpu to cuda)
G.cpu()
D.cpu()

def reset_grad():
    G.zero_grad()
    D.zero_grad()
    
#initialize optimizers and dataholders
G_solver = optim.RMSprop(G.parameters(), lr=lr)
D_solver = optim.RMSprop(D.parameters(), lr=lr)
G_losses = []
D_losses = []
data_iter = iter(train_loader)

In [ ]:
#training 
print("Start training WGAN model...")
d_iter = 2 
for it in range(500000):
    if it<100: d_iter = 50 
    else:
        d_iter = 2
    for _ in range(d_iter):
        # Sample data
        z = Variable(torch.randn(mb_size, z_dim)).cpu()
        try:
            X, _ = data_iter.next() 
        except StopIteration:
            data_iter = iter(train_loader) 
            X, _ = data_iter.next()
        #X = Variable(torch.from_numpy(X))
        X = X.cpu()
        
        # Dicriminator forward-loss-backward-update 
        G_sample = G(z)
        D_real = D(X)
        D_fake = D(G_sample)
        
        D_loss = -(torch.mean(D_real) - torch.mean(D_fake))
        
        D_loss.backward() 
        D_solver.step()
        
        # Weight clipping 
        for p in D.parameters():
            p.data.clamp_(-0.01, 0.01)
            
        D_losses.append(D_loss.data.cpu().numpy())
        
        # Housekeeping - reset gradient 
        reset_grad()
        
    # Generator forward-loss-backward-update 
    #X = Variable(torch.from_numpy(X)) 
    z = Variable(torch.randn(mb_size, z_dim)).cpu() #change cpu to cuda
    
    G_sample = G(z)
    D_fake = D(G_sample)
    
    G_loss = -torch.mean(D_fake)
    
    G_loss.backward() 
    G_solver.step()

    # Housekeeping - reset gradient 
    reset_grad()

    G_losses.append(G_loss.data.cpu().numpy())

    # Print and plot every now and then 
    if it % 1000 == 0:
        print('Iter-{}; D_loss: {}; G_loss: {}'
              .format(it, D_loss.data.cpu().numpy(),  
                G_loss.data.cpu().numpy()))
        
        samples = G(z).data.numpy()[:16]

#plot losses
plt.figure(figsize=(10,5))
plt.title("Generator and Critic Loss During Training") 
plt.plot(G_losses,label="Generator") 
plt.plot(D_losses,label="Critic") 
plt.xlabel("iter")
plt.ylabel("loss")
plt.legend()
plt.savefig('KDD_loss_2.png')

#save model
torch.save({'modelG_state_dict': G.state_dict(), 
            'modelD_state_dict': D.state_dict(),
            'optimizerG_state_dict': G_solver.state_dict(),
            'optimizerD_state_dict': D_solver.state_dict(),
            'G_loss': G_losses,
            'D_loss': D_losses
           }, "WGAN_model_KDD_2.pth")

# Data generation from saved WGAN model

In [ ]:
#loaded NN model
print("Loading model...")
print("----------------------------------------")
json_file = open("DNN_critic_model_KDD_2.json","r") 
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json) 
loaded_model.load_weights("DNN_critic_model_KDD_2.h5")

G = torch.nn.Sequential(
    torch.nn.Linear(z_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, h_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_dim, X_dim), 
    torch.nn.Sigmoid()
)

checkpoint = torch.load("WGAN_model_KDD_2.pth")
G.load_state_dict(checkpoint['modelG_state_dict'])

G.eval()
G.cpu()

final_Data = pd.DataFrame(columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'Label']) 
print("empty df",final_Data) 
print("----------------------------------------")
print("Completed loading model")

In [ ]:
it =0

while(final_Data.shape[0]<20000):
    it+=1
    z = Variable(torch.randn(100, z_dim)).cpu() 
    with torch.no_grad():
        data = G(z)
    data = data.cpu().numpy() 
    fake = pd.DataFrame(data)
        
    newy = np.ones((fake.shape[0],)) 
    #this is the model which changes 
    new_pred = loaded_model.predict(fake)
    new_pred = (new_pred > 0.5)
    cm = confusion_matrix(newy,new_pred)
    print("-------------------------------------------------------------") 
    print (" confusion_matrix for new data with deeper model \n" ,cm)
    
    combined_data = np.concatenate((fake,new_pred),axis=1) 
    combined_data = pd.DataFrame(combined_data)
    combined_data.columns = ['duration', 'protocol_type', 'service', 'flag', 'src_bytes',
       'dst_bytes', 'land', 'wrong_fragment', 'urgent', 'hot',
       'num_failed_logins', 'logged_in', 'num_compromised', 'root_shell',
       'su_attempted', 'num_root', 'num_file_creations', 'num_shells',
       'num_access_files', 'num_outbound_cmds', 'is_host_login',
       'is_guest_login', 'count', 'srv_count', 'serror_rate',
       'srv_serror_rate', 'rerror_rate', 'srv_rerror_rate', 'same_srv_rate',
       'diff_srv_rate', 'srv_diff_host_rate', 'dst_host_count',
       'dst_host_srv_count', 'dst_host_same_srv_rate',
       'dst_host_diff_srv_rate', 'dst_host_same_src_port_rate',
       'dst_host_srv_diff_host_rate', 'dst_host_serror_rate',
       'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
       'dst_host_srv_rerror_rate', 'Label']
    
    criteria1 = combined_data['Label'] == 1 #just attack data 
    data1 = combined_data[criteria1]
    print("only attack data - ",data1.shape) 
    final_Data = final_Data.append(data1, ignore_index = True) 
    print("final data ", it, "- ",final_Data.shape) 
    print("-------------------------------------------------------------") 
    
print(final_Data['Label'].value_counts())
print("---------------------------------------------------------")
df2 = final_Data.sample(n=13513, replace=False, random_state=1)
df3 = final_Data.sample(n=19494, replace=False, random_state=1)
final = pd.concat([mydt,df2], ignore_index = True,axis=0)
final['Label'].value_counts()
df3.to_csv("../dataset/KDD-2018/generated_r2l.csv", index = False, header = True)
final.to_csv("../dataset/KDD-2018/balanced_data_r2l.csv", index = False, header = True)